In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import textblob
from textblob import TextBlob, Word
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import nltk.corpus as Corpus
import numpy as np
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
import string
table = str.maketrans('', '', string.punctuation)
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.stem import wordnet
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
word_lem = WordNetLemmatizer()
from textblob import TextBlob

import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def read_dataset():
    data = pd.read_csv(r"/content/drive/MyDrive/data.csv", sep = '\t', header = None)
    data.columns = ['id', 'sentence']
    return data

In [ ]:
def sample_data(data, sample_value):
    data = data[:][:sample_value]
    return data

In [ ]:
print(read_dataset())

            id                                           sentence
0            1  I always waited for things to get better. I al...
1            2  I’ve had rope under my dresser for a while now...
2            3  I’m just burnt out I have nothing left. My gf ...
3            4  I want to commit suicide.. I feel like I have ...
4            5  It's like being thrown in a pit of suffering t...
...        ...                                                ...
122272  122273  Hey y'all. Not really sure what I'm doing but ...
122273  122274  I’m just wondering if there’s anyone like me t...
122274  122275  Amber you don't matter no one loves you. Why a...
122275  122276  Title explains it. I am going to sleep, and I ...
122276  122277  If you look at my old post then you’ll see wha...

[122277 rows x 2 columns]


In [ ]:
data = read_dataset()

In [ ]:
print(data)

            id                                           sentence
0            1  I always waited for things to get better. I al...
1            2  I’ve had rope under my dresser for a while now...
2            3  I’m just burnt out I have nothing left. My gf ...
3            4  I want to commit suicide.. I feel like I have ...
4            5  It's like being thrown in a pit of suffering t...
...        ...                                                ...
122272  122273  Hey y'all. Not really sure what I'm doing but ...
122273  122274  I’m just wondering if there’s anyone like me t...
122274  122275  Amber you don't matter no one loves you. Why a...
122275  122276  Title explains it. I am going to sleep, and I ...
122276  122277  If you look at my old post then you’ll see wha...

[122277 rows x 2 columns]


In [ ]:
def avg_word_len (sentence):
    words = sentence.split()
    avg_len = sum(len(word) for word in words)/len(words)
    return avg_len
def extract_ngrams(data, num):
    '''
    Function to generate n-grams from sentences
    '''
    n_grams = TextBlob(data).ngrams(num)
    return [ ' '.join(grams) for grams in n_grams]

In [ ]:
print(data.shape)
data.head()

(122277, 2)


,id,sentence
0,1,I always waited for things to get better. I al...
1,2,I’ve had rope under my dresser for a while now...
2,3,I’m just burnt out I have nothing left. My gf ...
3,4,I want to commit suicide.. I feel like I have ...
4,5,It's like being thrown in a pit of suffering t...


In [ ]:
data['char_count'] = data['sentence'].str.len()
df_train_sort_charcount = data.sort_values(by='char_count', ascending=False)
df_train_sort_charcount[['sentence', 'char_count']].head()

,sentence,char_count
6473,https://www.evernote.com/shard/s405/sh/cb9e84c...,39883.0
44562,"When I started writing this, I didn't realize ...",39384.0
8570,Get out of my head. Get out of my head. Get ou...,38719.0
39102,片翼の天使 片翼の天使 片翼の天使 片翼の天使 片翼の天使 片翼の天使 片翼の...,38287.0
93875,Some may remember me. I was asked what’s going...,37055.0


In [ ]:
data['word_count'] = data['sentence'].apply(lambda x: len(str(x).split(" ")))
df_train_sort_wordcount = data.sort_values(by='word_count', ascending=False)
df_train_sort_wordcount[['sentence','word_count']].head()

,sentence,word_count
39102,片翼の天使 片翼の天使 片翼の天使 片翼の天使 片翼の天使 片翼の天使 片翼の...,10940
8570,Get out of my head. Get out of my head. Get ou...,9680
44562,"When I started writing this, I didn't realize ...",7636
6473,https://www.evernote.com/shard/s405/sh/cb9e84c...,7115
94633,My name is Richard. I’m 28 years old. I was bo...,7088


In [ ]:
data = data['sentence'][0]
 
print("1-gram: ", extract_ngrams(data, 1))
print("2-gram: ", extract_ngrams(data, 2))
print("3-gram: ", extract_ngrams(data, 3))
print("4-gram: ", extract_ngrams(data, 4))

1-gram:  ['I', 'always', 'waited', 'for', 'things', 'to', 'get', 'better', 'I', 'always', 'waited', 'for', 'things', 'to', 'improve', 'and', 'people', 'to', 'change', 'and', 'for', 'myself', 'to', 'be', 'happy', 'I', '’', 'm', 'so', 'tired', 'of', 'waiting', 'Nothing', 'is', 'going', 'to', 'get', 'better', 'for', 'me', 'anyways', 'My', 'family', 'is', 'still', 'gon', 'na', 'be', 'shitty', 'tomorrow', 'My', 'past', 'isn', '’', 't', 'gon', 'na', 'change', 'tomorrow', 'My', 'brain', 'isn', '’', 't', 'going', 'to', 'change', 'tomorrow', 'The', 'way', 'I', '’', 'm', 'so', 'extremely', 'defective', 'and', 'unloveable', 'isn', '’', 't', 'going', 'to', 'change', 'tomorrow', 'And', 'the', 'tomorrow', '’', 's', 'are', 'interchangeable', 'for', 'any', 'range', 'of', 'time', 'as', 'I', '’', 've', 'been', 'waiting', 'for', 'all', 'of', 'this', 'for', 'years', 'now', 'I', 'can', '’', 't', 'remember', 'a', 'time', 'where', 'I', 'didn', '’', 't', 'want', 'to', 'give', 'up', 'I', 'just', 'am', 'so', 's

In [ ]:
data

'I always waited for things to get better. I always waited for things to improve and people to change and for myself to be happy. I’m so tired of waiting. Nothing is going to get better for me anyways. My family is still gonna be shitty tomorrow. My past isn’t gonna change tomorrow. My brain isn’t going to change tomorrow. The way I’m so extremely defective and unloveable isn’t going to change tomorrow. And the tomorrow’s are interchangeable for any range of time as I’ve been waiting for all of this for years now. I can’t remember a time where I didn’t want to give up. I just am so sick of this. I don’t know what I did to deserve it all. No matter what I do it’s not good enough and I’m not gonna be okay ever. I can’t see myself ever being older than this. I don’t want to do it anymore.'

In [ ]:
def convert_to_string (df_train):
    for i in range(len(df_train)):
        df_train['sentence'][i] = str(df_train['sentence'][i])
    return data

def data_processing(data):
    for i in range(len(data)):
        # Remove Excape Sequences
        sentence = remove_excape_sequesces(data['sentence'][i])
        # Remove Links
        sentence = remove_links(sentence)
        # Forming the final paragraphs
        senetnce = form_paragraph(sentence)
        # Perform further NLP processing
        sentence = apply_NLP(sentence)

        data['sentence'][i] = sentence
    return data

def apply_NLP (sentence):
    sentences = sentence.split(sep = '.')
    for i in range(len(sentences)):
        # Tokenize the words
        tokens = word_tokenize(sentences[i])
        tokens = [word.lower() for word in tokens]
        # Remove puntuations
        no_punctuations = [word.translate(table) for word in tokens]
        # Remove all non-alphabetic characters.
        words = [word for word in no_punctuations if word.isalpha()]
        # Remove Stop words and convert the sentence to its base form
        words = [word_lem.lemmatize(w) for w in words if not w in stop_words]
        sentences[i] = " ".join(words)
    sentence = " .".join(sentences)
    return sentence+"."

def remove_excape_sequesces (sentence):
    excape_sequences = ['\n', '\t', '\r', '\\', '\a', '\f', '\o', '\v', '\b', '\"', '\'', '\newline']
    temp = ""
    sentence = str(sentence)
    for i in sentence:
        if i in excape_sequences:
            continue
        temp += i
    return temp

def remove_links (sentence):
    comment = sentence.split(sep = " ")
    for word in comment:
        if ('http' in word) or ('www' in word) or ('https' in word) or (len(word) == 0):
            comment.remove(word)
    sentence = " ".join(comment)
    return sentence

def form_paragraph (sentence):
    temp = nltk.sent_tokenize(sentence)
    for index in range(len(temp)):
        temp[index] = temp[index].replace('.', ' ')
    for index in range(len(temp)):
        d = []
        k = temp[index].split(sep = ' ')
        for z in range(len(k)):
            if len(k[z]) > 0:
                d.append(k[z]) 
        s = ''
        for j in range(len(d)):
            s = s + ' ' + d[j].strip()
        temp[index] = s.strip()      
    sentence = '. '.join(temp).lower()
    return sentence

In [ ]:
def polarity_and_subjectivity(data):
    polarity_sentence  = []
    subjectivity_sentence = []

    for i in range(len(data)):
        
        # create temp variables
        sentence = data['sentence'][i]
        
        # form object and calculate polarity and subjectivity of sentence
        obj_sentence = TextBlob(sentence)
        polarity = obj_sentence.sentiment.polarity
        subjectivity = obj_sentence.sentiment.subjectivity
        
        # save the polarity and subjectivity of sentence
        polarity_sentence.append(polarity)
        subjectivity_sentence.append(subjectivity)

    data['polarity'] = polarity_sentence
    data['subjectivity'] = subjectivity_sentence
    
    return data

In [ ]:
def filter_polarity_and_subjectivity (data):
    k = []
    for i in range(len(data)):
        if data['subjectivity'][i] <= subjectivity_threshold and len(str(data['sentence'][i])) > 10:
            k.append(data['id'][i])
    data = data.drop(np.array(k)-1)
    labels = []
    data = data.reset_index(drop=True)
    for i in range(len(data)):
        if data['polarity'][i] < polarity_threshold:
            labels.append(1)
        else:
            labels.append(0)
    data['labels'] = labels
    return data

In [ ]:
data = read_dataset()
data = sample_data(data, 10000)
data = convert_to_string(data)
data = polarity_and_subjectivity(data)
polarity_threshold = 0          # range(-1 to +1)
subjectivity_threshold = 0.5    # range(0 to 1)
data = filter_polarity_and_subjectivity(data)
data = data_processing(data)
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,sentence,polarity,subjectivity,labels
0,1,always waited thing get better .always waited ...,0.057035,0.612771,0
1,6,tonight night dont care fucking hurt . . . ..,-0.600000,0.800000,1
2,7,im tucking tired people pretend care say theyr...,0.123889,0.582778,0
3,9,sucidal intention turned cutting stop many cut...,0.162500,0.558333,0
4,10,slowly becoming mentally unstable show.,0.337500,0.512500,0
...,...,...,...,...,...
5758,9990,get straight point .fucking close .close .purp...,-0.125000,0.518750,1
5759,9992,tried everything sure higher force preventing ...,0.166667,0.796296,0
5760,9993,somehow make want end life feeling .know end e...,0.800000,1.000000,0
5761,9995,given enough opportunity academic music spat f...,0.085714,0.533333,0


In [ ]:
data

,id,sentence,polarity,subjectivity,labels
0,1,always waited thing get better .always waited ...,0.057035,0.612771,0
1,6,tonight night dont care fucking hurt . . . ..,-0.600000,0.800000,1
2,7,im tucking tired people pretend care say theyr...,0.123889,0.582778,0
3,9,sucidal intention turned cutting stop many cut...,0.162500,0.558333,0
4,10,slowly becoming mentally unstable show.,0.337500,0.512500,0
...,...,...,...,...,...
5758,9990,get straight point .fucking close .close .purp...,-0.125000,0.518750,1
5759,9992,tried everything sure higher force preventing ...,0.166667,0.796296,0
5760,9993,somehow make want end life feeling .know end e...,0.800000,1.000000,0
5761,9995,given enough opportunity academic music spat f...,0.085714,0.533333,0


In [ ]:
data.shape


(5763, 5)

In [ ]:
data.sample(10)


,id,sentence,polarity,subjectivity,labels
3437,6041,would finally peace.,0.000000,1.000000,0
3957,6887,brain trauma joke amp scar family im selfish ....,0.062374,0.550505,0
2768,4850,tried killing hour ago failed .dont know im .p...,-0.489286,0.739286,1
679,1170,asleep instead im finding pushing everyone awa...,0.114394,0.575000,0
1771,3092,today good day .probably gon na jinx put line ...,-0.075000,0.716667,1
2499,4377,end life everything need .go peacefully pain m...,-0.194643,0.545635,1
2607,4586,got nothing better go .severely depressed good...,0.045572,0.505136,0
5275,9180,thought could finally something impressive med...,0.242857,0.800000,0
2507,4388,note amp good kid swear addiction .dont know k...,0.136166,0.616579,0
4708,8210,bipolar disorder people attacking .im harassed...,-0.140000,0.520000,1


In [ ]:
def preprocess_text(sentence):
    sentence = sentence.lower() # Convert to lowercase
    sentence = re.sub(r'[^\x00-\x7F]+',' ', sentence) # Remove words with non-ASCII characters
    words = sentence.split()
    words = filter(lambda x: x[0]!= '@' , sentence.split()) # Remove user tags
    words = [word for word in words if word not in set(stopwords.words('english'))] # Remove stop words
    sentence = " ".join(words)
    return sentence

In [ ]:
data['preprocessedsentence'] = data.sentence.apply(preprocess_text)
display(data.head())
data1 = data

,id,sentence,polarity,subjectivity,labels,preprocessedsentence
0,1,always waited thing get better .always waited ...,0.057035,0.612771,0,always waited thing get better .always waited ...
1,6,tonight night dont care fucking hurt . . . ..,-0.600000,0.800000,1,tonight night dont care fucking hurt . . . ..
2,7,im tucking tired people pretend care say theyr...,0.123889,0.582778,0,im tucking tired people pretend care say theyr...
3,9,sucidal intention turned cutting stop many cut...,0.162500,0.558333,0,sucidal intention turned cutting stop many cut...
4,10,slowly becoming mentally unstable show.,0.337500,0.512500,0,slowly becoming mentally unstable show.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
example1 = "/content/drive/MyDrive/glove.6B.50d.txt"
file1 = open(example1, "r")

In [ ]:
file1

<_io.TextIOWrapper name='/content/drive/MyDrive/glove.6B.50d.txt' mode='r' encoding='UTF-8'>

In [ ]:
embeddings_index = dict()
f = file1
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
f

<_io.TextIOWrapper name='/content/drive/MyDrive/glove.6B.50d.txt' mode='r' encoding='UTF-8'>

In [ ]:
data.isna().sum()

id                      0
sentence                0
polarity                0
subjectivity            0
labels                  0
preprocessedsentence    0
dtype: int64

In [ ]:
from keras.preprocessing.text import Tokenizer

max_length = data.preprocessedsentence.apply(lambda x: len(x.split())).max()

t = Tokenizer()
t.fit_on_texts(data.preprocessedsentence)
vocab_size = len(t.word_index) + 1
encoded_sentences = t.texts_to_sequences(data.preprocessedsentence)
padded_sentences = pad_sequences(encoded_sentences, maxlen=max_length, padding='post')
vocab_size = len(t.word_index) + 1

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Bidirectional, LSTM, Dropout, BatchNormalization
from keras.layers.embeddings import Embedding


In [ ]:
embedding_matrix = np.zeros((vocab_size, 50))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(padded_sentences, data.labels, test_size=0.1, stratify=data.labels)

In [ ]:
model_glove = Sequential()
model_glove.add(Embedding(vocab_size, 50, input_length=max_length, weights=[embedding_matrix], trainable=True))
model_glove.add(Bidirectional(LSTM(20, return_sequences=True)))
model_glove.add(Dropout(0.2))
model_glove.add(BatchNormalization())
model_glove.add(Bidirectional(LSTM(20, return_sequences=True)))
model_glove.add(Dropout(0.2))
model_glove.add(BatchNormalization())
model_glove.add(Bidirectional(LSTM(20)))
model_glove.add(Dropout(0.2))
model_glove.add(BatchNormalization())
model_glove.add(Dense(64, activation='relu'))
model_glove.add(Dense(64, activation='relu'))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_glove.fit(x_train, y_train, epochs = 5, batch_size = 64)

Epoch 1/5
82/82 [==============================] - 56s 682ms/step - loss: 0.6814 - accuracy: 0.5629
Epoch 2/5
82/82 [==============================] - 46s 564ms/step - loss: 0.6661 - accuracy: 0.5854
Epoch 3/5
82/82 [==============================] - 46s 565ms/step - loss: 0.6467 - accuracy: 0.6167
Epoch 4/5
82/82 [==============================] - 47s 572ms/step - loss: 0.5946 - accuracy: 0.6774
Epoch 5/5
82/82 [==============================] - 46s 558ms/step - loss: 0.5520 - accuracy: 0.7165


In [ ]:
model_glove.fit(x_test, y_test, epochs = 5)

In [ ]:
batch_size=64

gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 10
                        )
